## 12단계: 가변 길이 인수(개선 편)

> 이번 단계에서는 11단계의 함수를 개선해보겠습니다. \
첫 번째는 Add 클래스(혹은 다른 구체적인 함수 클래스)를 '사용하는 사람'을 위한 개선이고, \
두 번째는 '구현하는 사람'을 위한 개선입니다.

### 12.1 첫 번째 개선: 함수를 사용하기 쉽게

'사용하는 사람'을 위해 개선해보자. \
`그림 12-1`과 같이 리스트나 튜플을 거치지 않고 인수와 결과를 직접 주고받도록 수정한다.

<img src="images/그림 12-1.png" width=600/>

In [1]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x

In [ ]:
class Function:
    def __call__(self, *inputs):  # 별표(*)를 붙여 가변인자로 만든다.
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)


x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 12.2 두 번째 개선: 함수를 구현하기 쉽도록

이번엔 '구현하는 사람'을 위한 개선이다. \
`그림 12-2`와 같이 Add 클래스의 forward문 역시 입력도 변수를 직접 받고 결과도 변수를 직접 돌려주도록 개선하자. 

In [3]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)  # 별표(*)를 붙여 언팩(unpack)
        if not isinstance(ys, tuple):  # 튜플이 아닌 경우 추가 지원
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    
    # Add 클래스의 forward 구현이 쉬워짐짐
    def forward(self, x0, x1):
        y = x0 + x1
        return y

### 12.3 add 함수 구현

마지막으로 Add 클래스를 '파이썬 함수'로 사용할 수 있는 코드를 추가한다.

In [ ]:
def add(x0, x1):
    return Add()(x0, x1)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)  # Add 클래스 생성 과정을 숨김김
print(y.data)

5


지금까지 순전파에 가변 길이 인수를 적용했다. \
다음 단계부터는 '역전파'의 가변 길이 인수를 구현한다.